# Setup do ambiente

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc
from sklearn.ensemble import IsolationForest
import warnings

from sklearn import tree
import graphviz

from IPython.display import display

import pickle

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 7)

print("Bibliotecas importadas com sucesso!")

# Carregamento dos Dados

## Carregar o dataset


Carregar o dataset diretamente do [Github](https://github.com/diegonogare/Workshop-AnalisePreditivaDeManutencao/blob/main/conjuntoDados.csv).

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/diegonogare/Workshop-AnalisePreditivaDeManutencao/refs/heads/main/conjuntoDados.csv',
                 names=['ID', 'Timestamp', 'Temperatura', 'Sinais', 'Desligou'])

In [ ]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

df['Dia'] = df['Timestamp'].dt.day
df['DiaSemana'] = df['Timestamp'].dt.day_name()
df['Hora'] = df['Timestamp'].dt.time

df = df[df['Dia'] != (7)]
df = df[df['Dia'] != (29)]


## Conhecer o básico do dataset

Após o dataset ser carregado, é importante analisar seu conteúdo para conhecer o que há lá dentro.

In [ ]:
print("\nDataset carregado com sucesso!")
print("Formato do dataset (linhas x colunas):", df.shape)

print("\nPrimeiras 5 linhas do dataset:")
print(df.head())

print("\nInformações do dataset:")
df.info()

print("\nEstatística descritivas:")
print(df.describe())

print("\nValores nulos por Coluna:")
print(df.isnull().sum())

# Análise Exploratória Descritiva (EDA)

In [ ]:
colunas_numericas = df.select_dtypes(include=np.number).columns.tolist()
colunas_numericas

colunas_numericas.remove('ID') #ID da transação
colunas_numericas.remove('Desligou') #Flag se desligou ou não
colunas_numericas


In [ ]:
for col in colunas_numericas:
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f'Distribuição de {col}')
    plt.xlabel(col)
    plt.ylabel('Frequência')
    plt.show()
    print("\n")

In [ ]:
for col in colunas_numericas:
    sns.boxplot(x='Desligou', y=col, data=df)
    plt.title(f'{col} vs. Desligou')
    plt.show()
    print("\n")

In [ ]:
#Pela exploração que fizemos, apesar de Dia ser um valor numério, não tem relação com o objetivo que temos. Vamos remover
colunas_numericas.remove('Dia')

In [ ]:
correlation_matrix = df[colunas_numericas + ['Desligou']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlação')
plt.show()

### 1 - Quais horários a temperatura da fábrica é mais alta?

In [ ]:
temperatura = df.copy()
temperatura.drop(columns=['Timestamp', 'Dia', 'Sinais', 'Desligou'], inplace=True )

temperatura.groupby('Hora')['Temperatura'].mean('Temperatura').plot(kind='line', marker='')
plt.title('Temperatura Média por Hora do Dia')
plt.xlabel('Hora do Dia')
plt.ylabel('Temperatura Média (°C)')
plt.grid(True)
plt.show()


In [ ]:
for dia in df['Dia'].unique():
    temperaturaDiaria = df.copy()
    temperaturaDiaria = temperaturaDiaria[temperaturaDiaria['Dia'] == dia]
    temperaturaDiaria.drop(columns=['Timestamp', 'Sinais', 'Desligou'], inplace=True )
    diaSemana = temperaturaDiaria['DiaSemana'].iloc[0]

    temperaturaDiaria.groupby('Hora')['Temperatura'].mean('Temperatura').plot(kind='line', marker='')
    plt.title(f'Temperatura Média por Hora do {dia} - {diaSemana}')
    plt.xlabel('Hora do Dia')
    plt.ylabel('Temperatura Média (°C)')
    plt.grid(True)
    plt.show()

    print("\n")


### 2 - Quais temperaturas normalmente levam a máquina a parar de funcionar?


In [ ]:
maxFuncionando = df[df['Desligou'] == 0]['Temperatura'].max()
minNaoFuncionando = df[df['Desligou'] == 1]['Temperatura'].min()

sns.stripplot(x='Desligou', y='Temperatura', data=df, jitter=True, alpha=0.5)
plt.title('Temperatura vs. estado do equipamento (com Jitter)')
plt.xlabel('Máquina Parou (0 = Não, 1 = Sim)')
plt.ylabel('Temperatura (°C)')
plt.hlines(y=[maxFuncionando, minNaoFuncionando], xmin=-0.5, xmax=1.5, colors='red', linestyles='dashed')
plt.text(x=0.5, y=minNaoFuncionando - 1, s=minNaoFuncionando, color='red' )
plt.text(x=0.5, y=maxFuncionando + 0.5, s=maxFuncionando, color='red' )
plt.show()

In [ ]:
df['Sinais'].value_counts().sort_index().plot(kind='bar')
plt.title('Distribuição da quantidade de Sinais coletados')
plt.xlabel('Quantidade de Sinais')
plt.ylabel('Frequência')
plt.show()

In [ ]:
sns.boxplot(x='Sinais', y='Temperatura', hue='Desligou', data=df)
plt.title('Temperatura vs. sinais por estado do equipamento')
plt.xlabel('Sinais')
plt.ylabel('Temperatura (°C)')
plt.legend(title='Desligou')
plt.show()


# Classificação com Regressão Logística

In [ ]:
X = df[colunas_numericas] # Features
y = df['Desligou'] # Target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
print(f"Tamanho do Treino: {X_train.shape}, Tamanho do Teste: {X_test.shape}")


In [ ]:
# Escalonamento (importante para muitos modelos)
scaler = StandardScaler() # Create an instance of StandardScaler

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
seed = 42
log_reg = LogisticRegression(random_state=seed)
log_reg.fit(X_train_scaled, y_train)
print("Modelo treinado!")

In [ ]:
print("\nRealizando previsões e avaliando o modelo...")
y_pred = log_reg.predict(X_test_scaled)
y_proba = log_reg.predict_proba(X_test_scaled) # Probabilidades para a classe 1 (Desligou)

accuracy = accuracy_score(y_test, y_pred)
print(f"\nAcurácia do Modelo: {accuracy:.4f}")

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Previsto')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()

print(classification_report(y_test, y_pred))

In [ ]:
# Coeficientes
print(f"\nCoeficiente (Temperatura): {log_reg.coef_}")
print(f"Intercepto: {log_reg.intercept_}")


In [ ]:
# Visualização da Curva Sigmoide (para uma feature)
X_test_sorted_indices = np.argsort(X_test['Temperatura'])
X_test_sorted = X_test['Temperatura'].iloc[X_test_sorted_indices]
y_proba_lr_sorted = y_proba[X_test_sorted_indices, 1]

y_test_sorted = y_test.iloc[X_test_sorted_indices]


In [ ]:
maxFuncionando = df[df['Desligou'] == 0]['Temperatura'].max()
minNaoFuncionando = df[df['Desligou'] == 1]['Temperatura'].min()

jitter = 0.1
y_jitter = y_test_sorted + np.random.uniform(-jitter, jitter, size=len(y_test_sorted))

plt.scatter(X_test_sorted, y_jitter, color='gray', zorder=20, alpha=0.05, label='Dados Reais (Teste)')
plt.plot(X_test_sorted, y_proba_lr_sorted, color='red', lw=3, label='Curva Sigmoide Predita')

plt.ylabel('Probabilidade de Desligar / Estado Real')
plt.xlabel('Temperatura (°C)')
plt.title('Regressão Logística: Probabilidade de Desligar vs. Temperatura')
plt.legend()

plt.vlines(x=[minNaoFuncionando, maxFuncionando], ymin=-0.25, ymax=1.25, colors='red', linestyles='dashed')

plt.text(y=0.5, x=minNaoFuncionando - 1.5, s=minNaoFuncionando, color='red' )
plt.text(y=0.5, x=maxFuncionando + 0.5, s=maxFuncionando, color='red' )

plt.show()

In [ ]:
def predizerQuebra(Temperatura):
  input_scaled = scaler.transform(Temperatura)

  predicted_class = log_reg.predict(input_scaled)
  prediction_proba = log_reg.predict_proba(input_scaled)

  predicted_class = log_reg.predict(input_scaled)
  prediction_proba = log_reg.predict_proba(input_scaled)
  prob_desligou = prediction_proba[0][1]

  print(f"\nTemperatura informada: {Temperatura[0][0]}°C")
  print(f"Classe Predita: {'Desligou' if predicted_class == 1 else 'Não Desligou'}")
  print(f"Probabilidade de Desligar: {prob_desligou:.4f}")

  plotarPredicao(Temperatura[0][0], prob_desligou)

In [ ]:
def plotarPredicao(Temperatura, prob_desligou):
  X_test_sorted_indices = np.argsort(X_test['Temperatura'])
  X_test_sorted = X_test['Temperatura'].iloc[X_test_sorted_indices]
  y_proba_lr_sorted = y_proba[X_test_sorted_indices, 1]

  y_test_sorted = y_test.iloc[X_test_sorted_indices]

  jitter = 0.1
  y_jitter = y_test_sorted + np.random.uniform(-jitter, jitter, size=len(y_test_sorted))
  plt.scatter(X_test_sorted, y_jitter, color='gray', zorder=20, alpha=0.05, label='Dados Reais (Teste)')
  plt.plot(X_test_sorted, y_proba_lr_sorted, color='red', lw=3, label='Curva Sigmoide Predita')

  # Plote o ponto da temperatura de input e sua probabilidade predita
  plt.scatter(Temperatura, prob_desligou, color='green', s=100, zorder=25, label=f'Input ({Temperatura}°C)')

  plt.ylabel('Probabilidade de Desligar / Estado Real')
  plt.xlabel('Temperatura (°C)')
  plt.title('Regressão Logística: Probabilidade de Desligar vs. Temperatura com Input')
  plt.legend()
  plt.grid(True)
  plt.show()

In [ ]:
input_temperature = float(input("Por favor, digite a temperatura para previsão (ex: 25.5): "))
input_sinais = float(input("Por favor, digite a quantidade de Sinais para previsão (ex: 10): "))

novaTemperatura = np.array([[input_temperature, input_sinais]])

predizerQuebra(novaTemperatura)

In [ ]:
model_lr_filename = 'logistic_regression_model.pkl'
with open(model_lr_filename, 'wb') as file:
    pickle.dump(log_reg, file)
print(f"\nModelo de Regressão Logística salvo como '{model_lr_filename}'")

# Detecção de Anomalias com Isolation Forest

In [ ]:
df_anomalias_detection = df.copy()

In [ ]:
# Função para gerar anomalias sintéticas (exemplo simples)
def gerar_anomalias_sinteticas(df_input, n_anomalias):
    col_temp = 'Temperatura'
    col_param = 'Sinais'
    fator_temp = 2
    val_param_anomalo=20

    df_mod = df_input.copy()
    indices_anomalias = np.random.choice(df_mod.index, n_anomalias, replace=False)

    # Criar uma coluna para marcar anomalias sintéticas (apenas para nossa verificação)
    df_mod['anomaliaGerada'] = 0

    for idx in indices_anomalias:
        df_mod.loc[idx, col_temp] = df_mod.loc[idx, col_temp] * fator_temp # Anomalia de temperatura alta
        if np.random.rand() > 0.5: # Aleatoriamente, também mudar o Sinal
             df_mod.loc[idx, col_param] = val_param_anomalo # Um valor claramente fora do comum
        #df_mod.loc = 1
        df_mod.loc[idx, 'anomaliaGerada'] = 1

    return df_mod

In [ ]:
# Gerar algumas anomalias artificiais (~2% do df)
num_anomalias_artificiais = int(0.02 * len(df_anomalias_detection))
df_com_sinteticas = gerar_anomalias_sinteticas(df_anomalias_detection, num_anomalias_artificiais)

print(f"{num_anomalias_artificiais} anomalias artificiai geradas")
print(f"{len(df_com_sinteticas[df_com_sinteticas['anomaliaGerada'] == 1])} anomalias artificiais encontradas no dataset")
print("\nExemplo de anomalias artificiais:")
print(df_com_sinteticas[df_com_sinteticas['anomaliaGerada'] == 1].head())


In [ ]:
# Selecionar features para o Isolation Forest
features_if = ['Temperatura', 'Sinais']
X_if = df_com_sinteticas[features_if]

In [ ]:
# 'auto' = o algoritmo estima o % de contaminação | pode-se definir um valor (0.05 para 5%)
model_if = IsolationForest(n_estimators=100, contamination='auto', random_state=42)

df_com_sinteticas['anomaly_if_pred'] = model_if.fit_predict(X_if)
df_com_sinteticas['anomaly_score_if'] = model_if.decision_function(X_if)


In [ ]:
# Análise dos Resultados
anomalias_detectadas = df_com_sinteticas[df_com_sinteticas['anomaly_if_pred'] == -1]
print(f"\nTotal de anomalias detectadas pelo Isolation Forest: {len(anomalias_detectadas)}")

# Verificar quantas das anomalias sintéticas foram detectadas
print(f"\nAnomalias sintéticas detectadas: {num_anomalias_artificiais} de {len(anomalias_detectadas)}")

print(f"\nExemplos de anomalias naturais detectadas ({len(anomalias_detectadas[anomalias_detectadas['anomaliaGerada'] == 0])}):")
print(anomalias_detectadas[anomalias_detectadas['anomaliaGerada'] == 0].head())

print(f"\nExemplos de anomalias artificiais detectadas ({len(anomalias_detectadas[anomalias_detectadas['anomaliaGerada'] == 1])}):")
print(anomalias_detectadas[anomalias_detectadas['anomaliaGerada'] == 1].head())

In [ ]:
sns.scatterplot(x='Temperatura', y='Sinais', hue='anomaly_if_pred', data=df_com_sinteticas,
                palette={1: 'blue', -1: 'red'}, style='Desligou', sizes=(20, 200), alpha=0.5)
plt.title('Detecção de Anomalias com Isolation Forest')
plt.xlabel('Temperatura (°C)')
plt.ylabel('Quantidade de Sinais')
plt.legend(title='Anomalia = -1 \n', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# O índice em Python começa em 0 e o ID começa em 1, por isso há diferença entre os valores
anomalias_artificiais = anomalias_detectadas.copy()
anomalias_artificiais = anomalias_artificiais[anomalias_artificiais['anomaliaGerada'] == 1]

ponto_anomalo_exemplo_idx = np.random.randint(low=1, high=len(anomalias_artificiais), size=1)
ponto_anomalo_exemplo_features = anomalias_artificiais.iloc[ponto_anomalo_exemplo_idx].head()

print(f"\nIndice da anomalia selecionada: {ponto_anomalo_exemplo_idx}")
print(f"Features da anomalia selecionada: \n{ponto_anomalo_exemplo_features}")

In [ ]:
single_tree_estimator = model_if.estimators_
feature_names_for_tree = list(ponto_anomalo_exemplo_features.columns)
print(feature_names_for_tree)

In [ ]:
dot_data = tree.export_graphviz(single_tree_estimator[0],
                                out_file=None,
                                feature_names=features_if,
                                filled=True,
                                rounded=True,
                                special_characters=True,
                                impurity=False,
                                proportion=False,
                                max_depth=10)

In [ ]:
graph = graphviz.Source(dot_data, format="png")
print("\nVisualização da primeira iTree (profundidade limitada para exibição):")
graph.render("itree_visualization_ponto_anomalo")


#display(graph)

In [ ]:
model_if_filename = 'isolation_forest_model.pkl'
with open(model_if_filename, 'wb') as file:
    pickle.dump(model_if, file)
print(f"\nModelo Isolation Forest salvo como '{model_if_filename}'")

# Consumindo os modelos

In [ ]:
caminho_lr = "/content/" + model_lr_filename

with open(model_lr_filename, 'rb') as file:
  loaded_lr_model = pickle.load(file)
  print(f"Modelo '{model_lr_filename}' carregado com sucesso.")

In [ ]:
caminho_if = "/content/" + model_if_filename

with open(model_if_filename, 'rb') as file:
  loaded_if_model = pickle.load(file)
  print(f"Modelo '{model_if_filename}' carregado com sucesso.")


In [ ]:
def fazerPredicao(NovosDados):
  NovosDadosScaled = scaler.transform(NovosDados)

  pred_lr = loaded_lr_model.predict(NovosDadosScaled)
  proba_lr = loaded_lr_model.predict_proba(NovosDadosScaled)

  pred_if = loaded_if_model.predict(NovosDados) # Retorna 1 para inlier, -1 para outlier
  score_if = loaded_if_model.decision_function(NovosDados) # Scores de anomalia

  apresentarResultados(NovosDadosScaled, NovosDados, pred_lr, proba_lr, pred_if, score_if)

In [ ]:
def apresentarResultados(NovosDadosScaled, NovosDados, pred_lr, proba_lr, pred_if, score_if):

  print("\n--- Resultado da Regressão Logística ---")
  print(f"Para Entrada (escalonado): Temperatura = {NovosDadosScaled[0][0]}, Sinais = {NovosDadosScaled[0][1]}")
  if pred_lr == 1:
    print("Predição: A máquina TEM CHANCE DE DESLIGAR.")
    print(f"Probabilidade de desligar (classe 1): {proba_lr[0,1]:.4f}")
    print(f"Probabilidade de não desligar (classe 0): {proba_lr[0,0]:.4f}")
  else:
    print("Predição: A máquina TEM BAIXA CHANCE DE DESLIGAR.")
    print(f"Probabilidade de desligar (classe 1): {proba_lr[0,1]:.4f}")
    print(f"Probabilidade de não desligar (classe 0): {proba_lr[0,0]:.4f}")

  print("\n--- Resultado do Isolation Forest ---")
  print(f"Para Entrada: Temperatura = {NovosDados[0][0]}, Sinais = {NovosDados[0][1]}")
  if pred_if == -1:
    print("Predição: O ponto é considerado uma ANOMALIA.")
  else:
    print("Predição: O ponto é considerado NORMAL.")
  print(f"Score de Anomalia (decision_function): {score_if[0]:.4f} (valores menores/mais negativos indicam maior anomalia)")

  plotarPredicao(NovosDados[0][0], proba_lr[0][1])

In [ ]:
df_teste = df_com_sinteticas[['Temperatura', 'Sinais', 'Desligou', 'anomaliaGerada', 'anomaly_if_pred']].copy()
print("\nExemplos com detecção de anomalias")
print(df_teste[df_com_sinteticas['anomaly_if_pred'] == -1].head())
print("\nExemplos sem detecção de anomalias")
print(df_teste[df_com_sinteticas['anomaly_if_pred'] == 1].head())

In [ ]:
temp_input = float(input("Digite o valor da Temperatura (ex: 25.5): "))
param_input = int(input("Digite o valor do Sinais (ex: 10): "))

novaEntrada = np.array([[temp_input, param_input]])

fazerPredicao(novaEntrada)